# **Script**

# Classification_Model

## Load Model and Data Processing


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, mean_squared_error, mean_absolute_error
import keras as k
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn as sk


# Load the classification model
classification_model = tf.keras.models.load_model('model_classification.h5')

# print model summary
print("Model Summary:")
classification_model.summary()


# read the testing datrQuaa
excel_file_path = "diff.xlsx"
data = pd.read_excel(excel_file_path)

data.describe()

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 803 (3.14 KB)

 Trainable params: 801 (3.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

,Date,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,999,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,2004-09-29 15:09:32.972973056,-17.638739,1135.346430,-167.213213,9.991895,937.579997,218.771171,835.271021,93.472673,1463.869453,1029.596847,18.672047,50.212120,1.051753
min,2004-03-11 00:00:00,-200.000000,762.000000,-200.000000,0.710518,468.000000,-200.000000,329.750000,-200.000000,741.500000,326.000000,0.325000,9.875000,0.247748
25%,2004-05-17 12:00:00,0.900000,956.500000,-200.000000,4.418076,733.375000,95.000000,664.125000,78.150000,1252.625000,746.250000,13.112500,34.925000,0.829312
50%,2004-09-20 00:00:00,1.700000,1090.250000,-200.000000,8.272660,910.333333,172.000000,807.500000,107.000000,1466.750000,966.666667,17.100000,51.150001,1.012597
75%,2005-02-04 12:00:00,2.700000,1280.750000,-200.000000,13.662780,1105.625000,302.500000,974.750000,138.000000,1685.125000,1281.250000,23.875000,63.725000,1.223533
max,2005-04-04 00:00:00,9.400000,2039.750000,685.000000,43.943154,1851.000000,1247.000000,2095.250000,310.100000,2679.000000,2358.500000,40.900001,88.725000,2.066996
std,NaN,60.189143,229.710821,120.131022,7.311126,260.558397,225.718737,267.222901,88.312492,341.521758,380.201224,7.809003,18.256333,0.389671


### Classification data preprocessing

In [14]:
# Replace -200 values with NaN for further processing

linear_interpolate_data = data.replace(-200, np.nan)
columns_missing = linear_interpolate_data.columns[2:]
linear_interpolate_data[columns_missing] = linear_interpolate_data[columns_missing].interpolate(method='linear')

# calculate the mean value
co_gt_threshold = linear_interpolate_data['CO(GT)'].mean()
print(f"co threshold: {co_gt_threshold}")

linear_interpolate_data['CO_Target'] = (linear_interpolate_data['CO(GT)'] > co_gt_threshold).astype(int)

# ignore date, time and target column
correlation_matrix = linear_interpolate_data[linear_interpolate_data.columns[2:-1]].corr()

# ignore matrix based on correlation matrix
low_correlation_features = correlation_matrix['CO(GT)'].abs().sort_values().head(5).index.tolist()
print("Features with the lowest 5 correlation values relative to CO(GT):", low_correlation_features)


# feature selection
delete_column = ["CO(GT)", "Date", "Time", "CO_Target"]

delete_column.extend(low_correlation_features)
delete_column = list(set(delete_column))

x_value = linear_interpolate_data.drop(columns=delete_column)
y_value = linear_interpolate_data['CO_Target']

scaler = StandardScaler()
x_value = scaler.fit_transform(x_value)


x_value.shape, y_value.shape

co threshold: 2.0337337337337336
Features with the lowest 5 correlation values relative to CO(GT): ['RH', 'T', 'AH', 'NMHC(GT)', 'PT08.S3(NOx)']


((999, 7), (999,))

## Predict result

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, mean_squared_error, mean_absolute_error


# Prediction test set
y_pred = (classification_model.predict(x_value)>0.5).astype(int)

# get the confusion matrix
cm = confusion_matrix(y_value, y_pred)

tn, fp, fn, tp = cm.ravel()

accuracy = accuracy_score(y_value, y_pred)
precision = precision_score(y_value, y_pred)


print("")
print("Confusion Matrix:")
print("                   Positive (Predicted)     Negative (Predicted)")
print(f"Positive (Actual)     {tp:<28}{fp:<28}")
print(f"Negative (Actual)     {fn:<28}{tn:<28}")
print("")
print(f"{'':<30}{'Accuracy':<15}{'Precision':<15}{'Number of Samples'}")
print(f"CO(GT) classification{'':<10}{accuracy:.2%}{'':<10}{precision:.2%}{'':<12}{len(y_value)}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Confusion Matrix:
                   Positive (Predicted)     Negative (Predicted)
Positive (Actual)     346                         33                          
Negative (Actual)     46                          574                         

                              Accuracy       Precision      Number of Samples
CO(GT) classification          92.09%          91.29%            999


# Regression_Model

## Load Model and Data Processing

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, mean_squared_error, mean_absolute_error
import keras as k
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn as sk


# Load the trained model
regression_model = tf.keras.models.load_model('model_regression.h5')

# print model summary
print("Model Summary:")
regression_model.summary()



Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 803 (3.14 KB)

 Trainable params: 801 (3.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

### Regression data preprocessing

In [17]:

correlation_matrix = linear_interpolate_data[linear_interpolate_data.columns[2:-1]].corr()
# ignore matrix based on correlation matrix
low_correlation_features = correlation_matrix['NOx(GT)'].abs().sort_values().head(5).index.tolist()
print("Features with the lowest 5 correlation values relative to NOx(GT):", low_correlation_features)

delete_column = ["NOx(GT)", "Date", "Time", "CO_Target"]

delete_column.extend(low_correlation_features)

delete_column = list(set(delete_column))

x_value = linear_interpolate_data.drop(columns=delete_column)
y_value = linear_interpolate_data['NOx(GT)']

scaler = StandardScaler()
x_value = scaler.fit_transform(x_value)

x_value.shape

Features with the lowest 5 correlation values relative to NOx(GT): ['NMHC(GT)', 'AH', 'T', 'RH', 'PT08.S4(NO2)']


(999, 7)

## Predict result

In [18]:
# Calculate performance metrics
y_pred = regression_model.predict(x_value)

rmse = np.sqrt(mean_squared_error(y_value, y_pred))
mae = mean_absolute_error(y_value, y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Number of Samples: {len(y_value)}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Root Mean Squared Error (RMSE): 111.77102728302995
Mean Absolute Error (MAE): 65.06598826475586
Number of Samples: 999
